# Session 3a - Inmas Workshop Machine Learning Workshop, January 13-14, 2024

Instructor: Christian Kuemmerle - kuemmerle@uncc.edu

**This version of the notebook is more suitable for students with less experience in machine learning / who are less familiar with using Python for machine learning and/or the underlying concepts. It contains additional hints compared to the version ``session3a_K-means_advanced.ipynb``, but is similar in nature.**

## Clustering & K-means

In this exercise, we will implement k-means clustering on a simple 2D dataset to gain some intuition about how it works.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.cluster as cluster 
%matplotlib inline
sns.set_context('poster')
sns.set_color_codes()
plot_kwds = {'alpha' : 0.5, 's' : 80, 'linewidths':0}
import warnings
warnings.filterwarnings("ignore")

### Generating the dataset

We start with a toy dataset that consists of 3 clusters that are sampled from Gaussian distributions with different means and with the standard deviation 0.25.

In [ ]:
import sklearn.datasets as data
blobs, _ = data.make_blobs(n_samples=200, centers=[(-0.75,2.25), (1.0, 2.0), (0,1)], cluster_std=0.25)

fig, ax = plt.subplots()
ax.scatter(blobs.T[0], blobs.T[1], c='b', **plot_kwds)
ax.set_title('Toy Dataset', size=16)

plt.show();

## K-means

Despite its limitations, K-means is the 'go-to' clustering algorithm for many simply because it is fast, easy to understand, and available everywhere (there's an implementation in almost any statistical or machine learning tool you care to use). This is a brief summary of how the algorithm works:

### Optimization

- Formally, it's an optimization over the possible groupings of objects

> For a data set $X = \{ x_l \}_{l=1}^n$ of size $n$ where $x_l\in \mathbb{R}^d$ for all $l \in \{1,\ldots, n\}$:
><br>
><br>
>$\displaystyle  \hat{{C}} = \textrm{arg}\min_{{C}} \sum_{i=1}^k \left[\ \sum_{x\in{}C_i}\ \lvert\!\lvert x-\mu_i\rvert\!\rvert^2 \right]  =: \textrm{arg}\min_{{C}} F(C,\{\mu_i\}_{i=1}^k)$
><br>
><br>
> where 
><br>
><br>
>$\displaystyle  \mu_i = \frac{1}{\lvert{C_i}\rvert}\sum_{x\in{}C_i} x $.

The value of $F(C,\{\mu_i\}_{i=1}^k)$ is also called _distortion measure_ or _inertia_. 

Here, the $C = \{C_1,\ldots,C_k\}$ corresponds to a partition of the set of data points $X = \{ x_l \}_{l=1}^n$ of size $n$.

That means that $X = C_1 \cup C_2 \cup \ldots C_k$ with $C_i \cap C_j = \emptyset$ for $i \neq j \in \{1,2,\ldots,n\}$. 

**Note:** <br>
K-means can be generalized other distance notions than the squared Euclidean one $d(x,y) = \|x - y\|_2^2$, leading to better results if that distance notion conveys better the geometry of the data you want to cluster. However, in many cases, the computations involved will be more demanding, and the update step of the mean $\mu_i$ does not amount to averaging the cluster points anymore.

Cluster technique that fits into this framework: [$k$-medoid](https://en.wikipedia.org/wiki/K-medoids).

### Description of algorithm in words

- Iteratively improving the $\mu_i$ **prototypes** of $k$ clusters

>1. Pick $k$ random objects as the initial $\mu_i$ prototypes
>0. Find for each object the closest prototype and assign to that cluster
>0. Calculate the averages for each cluster to get new $\mu_i$
>0. Repeat until convergence


We use can use an implementation of k-means [provided by scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html). We visualize the resulting clustering (and cluster centers) below.

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, max_iter=200).fit(blobs)

fig, ax = plt.subplots()
ax.scatter(blobs.T[0], blobs.T[1], c=kmeans.labels_ , **plot_kwds)
ax.set_title('Toy Dataset', size=16)

C = kmeans.cluster_centers_
plt.scatter(C[:,0],C[:,1],c='r',marker='o',s=100,edgecolor='none');

plt.show();

Next, we will use the clustering_map function to show the separating hyperplanes that were learned by K-means. This shows how k-means would predict the clusters that new data points belong to.

In [ ]:
def clustering_map(X,cluster,i=0,j=1,h=0.005):
    '''
    h: step size in the mesh
    i: first feature number to be plotted
    j: second feature number to be plotted
    '''
    import matplotlib.pyplot as plt
    from matplotlib.colors import ListedColormap
    cmap_light = ListedColormap(['#FFBBBB', '#BBFFBB', '#BBBBFF'])
    cmap_bold = ListedColormap(['#CC0000', '#00AA00', '#0000CC'])
    # Points in a mesh of [x_min, m_max] x [y_min, y_max]
    x_min, x_max = X[:,i].min()-1, X[:,i].max()+1
    y_min, y_max = X[:,j].min()-1, X[:,j].max()+1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    grid = np.c_[xx.ravel(), yy.ravel()]
    
    # Obtain labels for each point in mesh. Use last trained model.
    cluster.fit(X)
    Z = cluster.predict(grid)
    
    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    fig = plt.figure()

    plt.pcolormesh(xx, yy, Z, cmap=cmap_light,shading='auto')
    
    # Plot the training points
    plt.scatter(X[:,i], X[:,j], **plot_kwds)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.title("Clustering with "+str(cluster))

    ax=plt.gca()
    #ax.legend(["training data"],loc=0,fontsize=8)
    
    # Plot the centroids as a white X
    centroids = cluster.cluster_centers_
    plt.scatter(centroids[:, 0], centroids[:, 1],
            marker='x', s=169, linewidths=3,
            color='w', zorder=10, alpha=0.8)
    
    return fig

In [ ]:
clustering_map(blobs,kmeans)
plt.show()

# Behavior of K-means for different data sets & weaknesses

The k-means algorithm is usually very fast but it has some weaknesses. It can be sensitive to the initialization, the scale and size of the different clusters. It also fails when the data cannot be separated by hyperplanes. In what follows, we will see some examples where k-means struggles to identify the correct clusters.

First, we will see how k-means is affected by the scale or standard deviation of the distributions from which the samples in the two clusters are drawn.

**After executing the following cell, change the parameter `cluster_std` of one of the blobs, and execute the cell again to see how this affects the geometry of the dataset.**

In [ ]:
import sklearn.datasets as data
blobs_0, _ = data.make_blobs(n_samples=300, centers=[(-0.75,2.25)], cluster_std=0.5)
blobs_1, _ = data.make_blobs(n_samples=300, centers=[(0.5, 2.0)], cluster_std=0.1)
data = np.vstack([blobs_0, blobs_1])
 
fig, ax = plt.subplots()
ax.scatter(data.T[0], data.T[1], c='b', **plot_kwds)
ax.set_title('Toy Dataset', size=16)

plt.show();

**Use K-means to fit this dataset. Visualize the result (dataset & cluster centers jointly) in a scatter plot.**

After you have done that, you can change the centers and standard deviations in "make_blobs" above and see how it affects the clustering.

<div class="alert-warning">
Hint: Feel free to use and revise the previous KMeans code (fourth cells above). Note that previously the data is called 'blobs', and here the data is called 'data'.
</div>

In [ ]:
### You can write your code here

## Behavior for imbalanced datasets

Now we see how k-means is affected by the size or number or samples in each cluster.

In [ ]:
import sklearn.datasets as data
nr_datapoints_blob0 = 20
nr_datapoints_blob1 = 500

blobs_0, _ = data.make_blobs(n_samples=nr_datapoints_blob0, centers=[(-0.5,2.25)], cluster_std=0.2)
blobs_1, _ = data.make_blobs(n_samples=nr_datapoints_blob1, centers=[(1, 2.0)], cluster_std=0.3)
data = np.vstack([blobs_0, blobs_1])
 
fig, ax = plt.subplots()
ax.scatter(data.T[0], data.T[1], c='b', **plot_kwds)
ax.set_title('Toy Dataset', size=16)

plt.show();

Note that we created the data set "blobs" such that one contains much fewer points than the other one.

**Use now K-means to fit this data set, and visualize the outcome coloring the datapoints with different colors based on their cluster affiliation. <br> You can change the centers and standard deviations and see how it affects the clustering.** 

In [ ]:
### You can write your code here

**Discuss with your group: Does K-means successfully "find" the two clusters? Discuss how the outcome might be related to the optimization objective and the fact how K-means works.**

Answer:

## Behavior for non-convex cluster geometries

Another interesting example is that of non-convex clusters. Consider the following example of a circle inside a ring.

In [ ]:
from sklearn import datasets

x, y = datasets.make_circles(n_samples=1000, factor=0.3, noise=0.1, random_state=2018)
plt.subplot(111, aspect='equal'); 
plt.scatter(x[:,0], x[:,1], c=y, **plot_kwds);

**Use K-means to find a clustering of this toy dataset and visualize the result.**

<div class="alert-warning">
Hint: Again, feel free to use and revise our previous K-means code. Note that here, the data is 'x'.
</div>

In [ ]:
### You can write your code here

We can see that k-means is unable to find the correct clustering. In general, k-means struggles when the the individual clusters cannot be separated by hyperplanes.

# Clustering Beyond K-Means

There is a variety of clustering algorithms that are better suited for such cases.

## Spectral Clustering
One example for such an algorithm is the Spectral Clustering algorithm. 

**Try to find a clustering using this scikit-learn implementation of the Spectral Clustering algorithm: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.SpectralClustering.html. Visualize the result.**

<div class="alert-warning">
Hint: the function that you need for Spectral Clustering algorithm is:
</div>
<br>
cluster.SpectralClustering(n_clusters=, affinity="nearest_neighbors").fit()

In [ ]:
### You can write your code here

### Color compression

One interesting application of clustering is in color compression within images. 
For example, imagine you have an image with millions of colors.
In most images, a large number of the colors will be unused, and many of the pixels in the image will have similar or even identical colors.

First, we plot the example image whose colors we would like to compress.

In [ ]:
from sklearn.datasets import load_sample_image
china = load_sample_image("china.jpg")

fig = plt.figure()
plt.imshow(china);

**Obtain a simplified 10-colored version of the image by above by applying k-means. Plot the resulting image and the original image next to each other.**

<div class="alert-warning">
Hint 1: to feed the image data to our KMeans function, you may need to reshape the image data first. You may use the code below.
</div>
<br>
im_rgb = np.array(china)
<br>
shp = im_rgb.shape
<br>
im_rgb = np.reshape(im_rgb, (shp[0]*shp[1], 3))

<div class="alert-warning">
Hint 2: After using KMeans to get the label of each pixel (i.e., kmeans.labels_), you need to change the color of each pixel to the corresponding color of its label, then we get the simplified version of this image. You can use the function written below.
</div>

In [ ]:
def average_colors(a, labels):
    #takes a flattened image a and labels, list of each pixel's cluster and return an image a_prime.
    #the color of a pixel of a_prime is the average color of all pixels sharing the same label.
    a_compressed = np.empty_like(a)
    for l in np.unique(labels):
        a_compressed[labels ==l, :] = np.mean(a[labels==l, :], 0)
    return a_compressed

In [ ]:
### You can write your code here